In [ ]:
# Este código analiza datos de clientes de Telecom X para entender por qué abandonan el servicio (churn)

## 1. Importación de Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import warnings
warnings.filterwarnings('ignore')

# Configuración para gráficos (estilo moderno y legible)
sns.set_style('whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 2. Carga de Datos desde la API
# URL de los datos en formato JSON
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

try:
    print("Conectando a la API de Telecom X...")
    response = requests.get(url)
    
    # Verificar si la conexión fue exitosa
    if response.status_code == 200:
        data = response.json()
        
        # Convertir los datos JSON a un DataFrame
        if isinstance(data, list):
            df = pd.DataFrame(data)
        elif isinstance(data, dict):
            possible_data_keys = [k for k in data.keys() if isinstance(data[k], list)]
            if possible_data_keys:
                df = pd.DataFrame(data[possible_data_keys[0]])
            else:
                df = pd.DataFrame([data])
        else:
            raise TypeError("Formato de datos no reconocido")
        
        print(f"Datos cargados correctamente. Hay {len(df)} registros.")
        
        # Desaplanar columnas con diccionarios anidados
        dict_columns = [col for col in df.columns if isinstance(df[col].iloc[0], dict)]
        
        if dict_columns:
            print(f"Desaplanando columnas: {dict_columns}")
            non_dict_columns = [col for col in df.columns if col not in dict_columns]
            new_df = df[non_dict_columns].copy()
            
            for col in dict_columns:
                nested_df = pd.json_normalize(df[col])
                nested_df.columns = [f"{col}_{subcol}" for subcol in nested_df.columns]
                new_df = pd.concat([new_df, nested_df], axis=1)
            
            df = new_df
            print(f"DataFrame ahora tiene {len(df.columns)} columnas")
        
    else:
        print(f"Error al conectar. Código: {response.status_code}")
except Exception as e:
    print(f"Error al procesar datos: {e}")

# Mostrar las primeras filas
print("\nPrimeras 5 filas del DataFrame:")
print(df.head())

## 3. Exploración Inicial de los Datos
# Información general
print("\nInformación del DataFrame:")
df.info()

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df.describe())

# Lista de columnas
print("\nColumnas del DataFrame:")
print(df.columns.tolist())

## 4. Diccionario de Datos
"""
- customerID: Identificador único del cliente
- gender: Género (Male/Female)
- SeniorCitizen: Persona mayor (1 = Sí, 0 = No)
- Partner: Tiene pareja (Yes/No)
- Dependents: Tiene dependientes (Yes/No)
- tenure: Meses con la empresa
- PhoneService: Servicio telefónico (Yes/No)
- MultipleLines: Múltiples líneas (Yes/No/No phone service)
- InternetService: Tipo de internet (DSL, Fiber optic, No)
- OnlineSecurity: Seguridad en línea (Yes/No/No internet service)
- OnlineBackup: Respaldo en línea (Yes/No/No internet service)
- DeviceProtection: Protección de dispositivos (Yes/No/No internet service)
- TechSupport: Soporte técnico (Yes/No/No internet service)
- StreamingTV: Streaming de TV (Yes/No/No internet service)
- StreamingMovies: Streaming de películas (Yes/No/No internet service)
- Contract: Tipo de contrato (Month-to-month, One year, Two year)
- PaperlessBilling: Facturación electrónica (Yes/No)
- PaymentMethod: Método de pago
- MonthlyCharges: Cargo mensual
- TotalCharges: Cargos totales
- Churn: Abandono del cliente (Yes/No)
"""

## 5. Verificación de Problemas en los Datos
# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Verificar duplicados en customerID
if 'customerID' in df.columns:
    duplicados_id = df['customerID'].duplicated().sum()
    print(f"\nDuplicados por customerID: {duplicados_id}")
    if duplicados_id > 0:
        print("Eliminando duplicados...")
        df = df.drop_duplicates(subset=['customerID'], keep='first')
else:
    print("\nNo se encontró la columna customerID")

# Verificar valores únicos en columnas categóricas
print("\nValores únicos en columnas categóricas:")
for col in df.select_dtypes(include=['object']).columns:
    unique_vals = df[col].unique()
    if len(unique_vals) > 10:
        print(f"{col}: {unique_vals[:5]}... (total: {len(unique_vals)} valores)")
    else:
        print(f"{col}: {unique_vals}")

# Verificar formato de account_Charges.Total
total_charges_col = 'account_Charges.Total'
if total_charges_col in df.columns:
    print(f"\nTipo de datos de {total_charges_col}: {df[total_charges_col].dtype}")
    if df[total_charges_col].dtype == 'object':
        print(f"Valores únicos en {total_charges_col}: {df[total_charges_col].unique()[:5]}")
else:
    print(f"\nError: No se encontró {total_charges_col}")

## 6. Limpieza y Transformación de Datos
# Convertir account_Charges.Total a numérico
if total_charges_col in df.columns:
    if df[total_charges_col].dtype == 'object':
        problematicas = pd.to_numeric(df[total_charges_col], errors='coerce').isna()
        print(f"\nFilas con valores no numéricos en {total_charges_col}: {problematicas.sum()}")
        if problematicas.sum() > 0:
            print("Valores problemáticos:")
            print(df.loc[problematicas, total_charges_col])
        
        df[total_charges_col] = pd.to_numeric(df[total_charges_col], errors='coerce')
        
        # Imputar valores nulos con la mediana
        if df[total_charges_col].isnull().sum() > 0:
            df[total_charges_col].fillna(df[total_charges_col].median(), inplace=True)
            print(f"Imputados valores nulos en {total_charges_col} con la mediana")
else:
    print(f"Error: No se encontró {total_charges_col}")

# Convertir SeniorCitizen a No/Yes
if 'customer_SeniorCitizen' in df.columns:
    if df['customer_SeniorCitizen'].dtype in ['int64', 'int32', 'float64']:
        df['customer_SeniorCitizen'] = df['customer_SeniorCitizen'].map({0: 'No', 1: 'Yes'})
        print("Columna customer_SeniorCitizen convertida a No/Yes")
else:
    print("No se encontró la columna customer_SeniorCitizen")

# Verificar valores nulos después de la limpieza
print("\nValores nulos después de la limpieza:")
print(df.isnull().sum())

## 7. Creación de la columna "Cuentas_Diarias"
monthly_charges_col = 'account_Charges.Monthly'
if monthly_charges_col in df.columns:
    df['Cuentas_Diarias'] = df[monthly_charges_col] / 30
    print("\nColumna 'Cuentas_Diarias' creada")
    print(df[[monthly_charges_col, 'Cuentas_Diarias']].head())
else:
    print(f"Error: No se encontró {monthly_charges_col}")

## 8. Estandarización y Transformación
# Convertir columnas binarias a 1/0
binary_columns = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 
                  'account_PaperlessBilling', 'Churn']
for col in binary_columns:
    if col in df.columns:
        df[col + '_Binary'] = df[col].map({'Yes': 1, 'No': 0})
        if df[col + '_Binary'].isnull().sum() > 0:
            print(f"Advertencia: Valores nulos en {col}_Binary. Imputando con 0...")
            df[col + '_Binary'].fillna(0, inplace=True)

# Traducir nombres de columnas al español
column_translation = {
    'customerID': 'ID_Cliente',
    'Churn': 'Evasion',
    'customer_gender': 'Genero',
    'customer_SeniorCitizen': 'ClienteMayor',
    'customer_Partner': 'Pareja',
    'customer_Dependents': 'Dependientes',
    'customer_tenure': 'Permanencia',
    'phone_PhoneService': 'ServicioTelefono',
    'phone_MultipleLines': 'LineasMultiples',
    'internet_InternetService': 'ServicioInternet',
    'internet_OnlineSecurity': 'SeguridadEnLinea',
    'internet_OnlineBackup': 'RespaldoEnLinea',
    'internet_DeviceProtection': 'ProteccionDispositivo',
    'internet_TechSupport': 'SoporteTecnico',
    'internet_StreamingTV': 'StreamingTV',
    'internet_StreamingMovies': 'StreamingPeliculas',
    'account_Contract': 'Contrato',
    'account_PaperlessBilling': 'FacturacionElectronica',
    'account_PaymentMethod': 'MetodoPago',
    'account_Charges.Monthly': 'CargoMensual',
    'account_Charges.Total': 'CargosTotal',
    'Cuentas_Diarias': 'CuentasDiarias'
}

df_es = df.copy()
df_es = df_es.rename(columns=column_translation)
print("\nDataFrame con nombres en español:")
print(df_es.head())

## 9. Análisis Descriptivo
# Estadísticas de variables numéricas
numeric_columns = ['customer_tenure', 'account_Charges.Monthly', 'account_Charges.Total', 'Cuentas_Diarias']
print("\nEstadísticas de variables numéricas:")
print(df[numeric_columns].describe())

# Distribución de Churn por variables categóricas
if all(col in df.columns for col in ['customer_gender', 'account_Contract', 'internet_InternetService', 'Churn']):
    print("\nDistribución de Churn por Género:")
    print(df.groupby('customer_gender')['Churn'].value_counts(normalize=True).unstack())
    
    print("\nDistribución de Churn por Contrato:")
    print(df.groupby('account_Contract')['Churn'].value_counts(normalize=True).unstack())
    
    print("\nDistribución de Churn por Servicio de Internet:")
    print(df.groupby('internet_InternetService')['Churn'].value_counts(normalize=True).unstack())

# Estadísticas por Churn
print("\nEstadísticas por Churn:")
print(df.groupby('Churn')[numeric_columns].describe())

## 10. Visualizaciones
# Función para graficar variables categóricas
def plot_categorical_churn(df, column, title, xlabel, rotation=0):
    if column in df.columns:
        plt.figure(figsize=(10, 6))
        sns.countplot(x=column, hue='Churn', data=df, palette=['#4CAF50', '#F44336'])
        plt.title(title, fontsize=14)
        plt.xlabel(xlabel)
        plt.ylabel('Número de Clientes')
        plt.legend(title='Churn')
        plt.xticks(rotation=rotation)
        plt.show()
    else:
        print(f"Error: La columna {column} no existe")

# Gráfico de distribución de Churn
if 'Churn' in df.columns:
    plt.figure(figsize=(8, 6))
    churn_counts = df['Churn'].value_counts()
    plt.pie(churn_counts, labels=churn_counts.index, autopct='%1.1f%%', startangle=90, colors=['#4CAF50', '#F44336'])
    plt.title('Distribución de Evasión (Churn)', fontsize=14)
    plt.axis('equal')
    plt.show()

# Gráficos de Churn por variables categóricas
plot_categorical_churn(df, 'customer_gender', 'Churn por Género', 'Género')
plot_categorical_churn(df, 'account_Contract', 'Churn por Tipo de Contrato', 'Tipo de Contrato')
plot_categorical_churn(df, 'internet_InternetService', 'Churn por Servicio de Internet', 'Servicio de Internet')
plot_categorical_churn(df, 'account_PaymentMethod', 'Churn por Método de Pago', 'Método de Pago', rotation=45)

# Distribución de Permanencia por Churn
if 'customer_tenure' in df.columns and 'Churn' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x='customer_tenure', hue='Churn', multiple='stack', palette=['#4CAF50', '#F44336'], bins=20)
    plt.title('Distribución de Permanencia por Churn', fontsize=14)
    plt.xlabel('Permanencia (meses)')
    plt.ylabel('Número de Clientes')
    plt.legend(title='Churn')
    plt.show()

# Distribución de Cargos Mensuales por Churn
if monthly_charges_col in df.columns and 'Churn' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=monthly_charges_col, hue='Churn', multiple='stack', palette=['#4CAF50', '#F44336'], bins=20)
    plt.title('Distribución de Cargos Mensuales por Churn', fontsize=14)
    plt.xlabel('Cargos Mensuales ($)')
    plt.ylabel('Número de Clientes')
    plt.legend(title='Churn')
    plt.show()

# Relación entre Permanencia y Cargos Mensuales
if all(col in df.columns for col in ['customer_tenure', monthly_charges_col, 'Churn']):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='customer_tenure', y=monthly_charges_col, hue='Churn', data=df, palette=['#4CAF50', '#F44336'], alpha=0.7)
    plt.title('Permanencia vs Cargos Mensuales por Churn', fontsize=14)
    plt.xlabel('Permanencia (meses)')
    plt.ylabel('Cargos Mensuales ($)')
    plt.legend(title='Churn')
    plt.show()

## 11. Análisis de Correlación
# Preparar DataFrame para correlación
df_corr = df.copy()

# Convertir Churn a binario
if 'Churn' in df_corr.columns:
    print("\nValores únicos en Churn:", df_corr['Churn'].unique())
    df_corr['Churn_Binary'] = df_corr['Churn'].map({'Yes': 1, 'No': 0})
    if df_corr['Churn_Binary'].isnull().sum() > 0:
        print(f"Eliminando {df_corr['Churn_Binary'].isnull().sum()} filas con Churn nulo...")
        df_corr = df_corr.dropna(subset=['Churn_Binary'])

# Convertir variables categóricas a dummies
cat_cols = ['customer_gender', 'customer_SeniorCitizen', 'customer_Partner', 'customer_Dependents', 
            'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService', 
            'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection', 
            'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies',
            'account_Contract', 'account_PaperlessBilling', 'account_PaymentMethod']

non_binary_cat = [col for col in cat_cols if col in df_corr.columns and df_corr[col].nunique() > 2]
df_corr = pd.get_dummies(df_corr, columns=non_binary_cat, drop_first=True)

# Convertir columnas binarias a 1/0
binary_map = {'Yes': 1, 'No': 0}
for col in ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']:
    if col in df_corr.columns:
        df_corr[col] = df_corr[col].map(binary_map)
        if df_corr[col].isnull().sum() > 0:
            print(f"Imputando valores nulos en {col} con 0...")
            df_corr[col].fillna(0, inplace=True)

# Seleccionar columnas para correlación
num_cols = ['customer_tenure', 'account_Charges.Monthly', 'account_Charges.Total', 'Cuentas_Diarias', 'Churn_Binary']
num_cols = [col for col in num_cols if col in df_corr.columns]
binary_dummies = [col for col in df_corr.columns 
                 if col not in cat_cols 
                 and col != 'customerID' 
                 and col not in num_cols
                 and df_corr[col].dtype in ['int64', 'float64', 'bool', 'uint8']]
corr_cols = num_cols + binary_dummies

# Verificar datos antes de correlación
for col in corr_cols:
    if col in df_corr.columns and df_corr[col].isnull().sum() > 0:
        print(f"Imputando {df_corr[col].isnull().sum()} valores nulos en {col} con 0...")
        df_corr[col].fillna(0, inplace=True)

# Calcular y visualizar correlación
try:
    corr_matrix = df_corr[corr_cols].corr()
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Matriz de Correlación', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    churn_corr = corr_matrix['Churn_Binary'].sort_values(ascending=False)
    print("\nCorrelación con Churn:")
    print(churn_corr)
    
    plt.figure(figsize=(10, 6))
    churn_corr_filtered = churn_corr[abs(churn_corr) > 0.05]
    sns.barplot(x=churn_corr_filtered.valuesotted_values, y=churn_corr_filtered.index, palette='viridis')
    plt.title('Correlaciones con Churn', fontsize=14)
    plt.xlabel('Coeficiente de Correlación')
    plt.ylabel('Variables')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()
except Exception as e:
    print(f"Error en la correlación: {e}")
    print("Columnas disponibles:", df_corr.columns.tolist())

## 12. Informe Final
print("\n=== INFORME FINAL ===")
print("""
# Análisis de Evasión de Clientes en Telecom X

## Introducción
Este proyecto analiza por qué los clientes de Telecom X abandonan el servicio (churn). 
Entender estos factores ayudará a la empresa a retener más clientes.

## Limpieza de Datos
1. Se cargaron datos desde una API en formato JSON.
2. Se desaplanaron columnas anidadas.
3. Se creó la columna 'Cuentas_Diarias' dividiendo cargos mensuales por 30.
4. Se convirtieron variables categóricas a numéricas para correlaciones.

## Resultados Principales
- **Tasa de Churn**: Aproximadamente el 26.5% de los clientes abandonan.
- **Contrato**: Los contratos mensuales tienen mayor churn (42.7%) que los de uno o dos años.
- **Internet**: Los clientes con fibra óptica tienen alto churn (41.9%).
- **Método de Pago**: El cheque electrónico está asociado con más churn (45.3%).
- **Permanencia**: Clientes nuevos (<12 meses) son más propensos a irse.
- **Cargos Mensuales**: Cargos altos aumentan la probabilidad de churn.

## Conclusiones
1. Los contratos mensuales son un gran factor de riesgo para el churn.
2. La fibra óptica tiene problemas que aumentan el abandono.
3. Los primeros meses son clave para retener clientes.
4. Los servicios adicionales (seguridad, soporte) reducen el churn.

## Recomendaciones
1. Ofrecer descuentos en contratos largos.
2. Mejorar la calidad del servicio de fibra óptica.
3. Crear un programa de bienvenida para clientes nuevos.
4. Promocionar servicios de seguridad y soporte técnico.
5. Investigar por qué el cheque electrónico tiene alto churn.

Con estas acciones, Telecom X puede reducir el churn y mejorar la retención.
""")

# Mostrar conclusiones finales
print("\n=== CONCLUSIONES ===")
print("1. Contratos mensuales tienen mayor riesgo de abandono.")
print("2. La fibra óptica está asociada con más churn.")
print("3. Los primeros 12 meses son críticos.")
print("4. Clientes con cheque electrónico tienden a irse más.")
print("5. Servicios de soporte y seguridad reducen el churn.")